# VAE with do notation

In [84]:
from tqdm import tqdm
import time
import argparse
import os
import sys
from abc import abstractmethod
from torch.nn import functional as F
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [2]:
import pandas as pd
import numpy as np

In [3]:
import torch
from torch import nn
import torch.nn.init as init
from torchvision import datasets
from torchvision import transforms as T
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from PIL import Image

In [4]:
sys.path.insert(0,"/cmlscratch/margot98/Causal_Disentangle/3dshape_dataset")

In [5]:
from datasets import ShapeDataset,CelebaDataset
from dataset_3d_shape import sample_3dshape_dataset

In [6]:
# data_path = "/cmlscratch/margot98/Causal_Disentangle/3dshape_data"
# attr_path =  "/cmlscratch/margot98/Causal_Disentangle/3dshape_data/sample_source.csv"

data_path = "/fs/cml-datasets/CelebA/Img/img_align_celeba"
attr_path =  "/fs/cml-datasets/CelebA/Anno/list_attr_celeba.txt"

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [91]:
image_size = 64
transform = T.Compose([
    T.Resize([image_size, image_size]),
    T.ToTensor(),
    T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# train_dataset = ShapeDataset( data_path='/cmlscratch/margot98/Causal_Disentangle/3dshape_data', 
#                               attr_path='/cmlscratch/margot98/Causal_Disentangle/3dshape_data/sample_source.csv',
#                               attr=['floor_hue', 'shape'],
#                               transform=transform
#                              )
train_dataset = train_dataset = CelebaDataset(data_path,attr_path,
                              attr=['Eyeglasses'],
                              transform=transform)
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=1024,
                          shuffle=True) 

In [70]:
def kaiming_init(m):
    if isinstance(m, (nn.Linear, nn.Conv2d)):
        init.kaiming_normal_(m.weight)
        if m.bias is not None:
            m.bias.data.fill_(0)
    elif isinstance(m, (nn.BatchNorm1d, nn.BatchNorm2d)):
        m.weight.data.fill_(1)
        if m.bias is not None:
            m.bias.data.fill_(0)
            
class View(nn.Module):
    def __init__(self, size):
        super(View, self).__init__()
        self.size = size

    def forward(self, tensor):
        return tensor.view(self.size)

In [71]:
class doVAE(nn.Module):

    def __init__(self,z_dim, c_dim = 2, in_channels=3):
        super(doVAE, self).__init__()
        
        self.z_dim = z_dim
        self.c_dim = c_dim
        # encoder output dim: c_Num*z_dim for mu_z, c_Num*z_dim for sigma_z, c_Num for mu_pi, c_Num for sigma_mu
        self.model_encoder= nn.Sequential(
            # B,  32, 32, 32
            nn.Conv2d(in_channels, out_channels=32, kernel_size=4, stride=2, padding=1),          
            nn.BatchNorm2d(32),
            nn.ReLU(),
            # B,  32, 16, 16
            nn.Conv2d(32, out_channels=32, kernel_size=4, stride=2, padding=1),  
            nn.BatchNorm2d(32),
            nn.ReLU(),
            # B,  64,  8,  8
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1),  
            nn.BatchNorm2d(64),
            nn.ReLU(),
            # B,  64,  4,  4
            nn.Conv2d(64, 64, kernel_size=4, stride=2, padding=1),          
            nn.BatchNorm2d(64),
            nn.ReLU(),
            # B, 256,  1,  1
            nn.Conv2d(64, 256, kernel_size=4, stride=1),           
            nn.BatchNorm2d(256),
            nn.ReLU(),
            View((-1, 256*1*1)),                 # B, 256
            
            # for mu and logvar
            nn.Linear(256,  c_dim*(2*z_dim+2)),      
        )
        
            
        self.model_decoder = nn.Sequential(
            nn.Linear(z_dim, 256),               # B, 256
            View((-1, 256, 1, 1)),               # B, 256,  1,  1
            nn.ReLU(),
            nn.ConvTranspose2d(256, 64, 4),      # B,  64,  4,  4
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 64, 4, 2, 1), # B,  64,  8,  8
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, 2, 1), # B,  32, 16, 16
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 32, 4, 2, 1), # B,  32, 32, 32
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.ConvTranspose2d(32, in_channels, 4, 2, 1),  # B, 3, 64, 64
        )
        

        self.weight_init()
        
    def weight_init(self):
        for block in self._modules:
            for m in self._modules[block]:
                kaiming_init(m)
        
                
    def encode(self, X):
        
        result = self.model_encoder(X)
        X_len = X.shape[0]
        mu = torch.zeros(X_len,self.c_dim * self.z_dim)
        logvar = torch.zeros(X_len,self.c_dim * self.z_dim)
        mu_pi = torch.zeros(X_len,self.c_dim)
        var_pi = torch.zeros(X_len,self.c_dim)
        for c in range(self.c_dim):
            mu[:,c*self.z_dim:(c+1)*self.z_dim] = result[:,c*self.z_dim : (c+1)*self.z_dim]
            logvar[:,c*self.z_dim:(c+1)*self.z_dim] = result[:,(self.c_dim+c)*self.z_dim : (self.c_dim+c+1)*self.z_dim]
            mu_pi[:,c] = result[:,2*self.c_dim*self.z_dim + c]
            var_pi[:,c] = result[:,2*self.c_dim*self.z_dim + self.c_dim + c]
                                
        
        return mu, logvar, mu_pi, var_pi
                                 
    
    def decode(self, z):
        
        return self.model_decoder(z)
    
    
    def generate(self, num_samples, current_device):                         
        z = torch.randn(num_samples, self.z_dim)
        z = z.to(current_device)
        samples = self.decode(z)
        return samples
    

    def reparameterize(self, mu, logvar,mu_pi,var_pi):
        X_len = mu.shape[0]
        z_all = torch.zeros(X_len, self.z_dim)
        Pi = torch.zeros(X_len, self.c_dim)
        for c in range(self.c_dim):
            std = torch.exp(0.5 * logvar[:,c*self.z_dim:(c+1)*self.z_dim])
            eps = torch.randn_like(std)
            curr_z = eps * std + mu[:,c*self.z_dim:(c+1)*self.z_dim]
            
            std_pi = torch.exp(0.5 * var_pi[:,c])
            eps_pi = torch.randn_like(std_pi)
            curr_pi = eps_pi * std_pi + mu_pi[:,c]
            
            z_all += curr_z * curr_pi[:,None]
            Pi[:,c] = curr_pi
        
        return z_all, Pi
    
    

    def forward(self, X):
        mu, logvar, mu_pi, var_pi = self.encode(X)
        z,Pi = self.reparameterize(mu, logvar, mu_pi, var_pi)
        reconstructed_z = self.decode(z)
        return reconstructed_z, mu, logvar, Pi
    
    def calculate_loss(self, X, reconstructed_z, mu, logvar, Pi, label, pi_weight = 1.0, kl_weight = 0.5):
        """
        Loss term currently contains: reconstruction loss, classification loss for p(c|x)
        """
        # classification loss:
        criterion = nn.CrossEntropyLoss()
        pi_loss = criterion(Pi, label.float())
        
        # reconstruction loss:
        rec_loss = F.mse_loss(reconstructed_z, X)
        
        kld_loss = 0
        for c in range(self.c_dim):
            # regulate to be N(mu_c, I), where mu_c learnt from NN
            log_var_c = logvar[:,c*self.z_dim:(c+1)*self.z_dim]
            kld_loss += torch.mean(-0.5 * torch.sum(1 + log_var_c - log_var_c.exp(), dim = 1), dim = 0)
        
        loss = rec_loss + pi_weight * pi_loss + kl_weight * kld_loss
        
        return loss, rec_loss, pi_loss, kld_loss
        
    
#     def calculate_loss(self, x1, x2):
#         """
#         Here we compute the distance d(E[Z_i|do(Z_{-i}), X], E[Z_i|X])
#         E[Z_i | X] = \sum_{i}f(x_i) /N = \sum_{c} P(C = c)E[Z_i|X, C = c]
#         E[Z_i | do(Z_{-i}), X] = \sum_{c}P(C = c)E[Z_i|Z_{-i},X, C = c]
        
#         Computation:
#         sum_c P(C = c)\sum_i d(E[Z_i|do(Z_{-i}), X, C = c], E[Z_i|X, C = c])
#         """
#         prob_c = []
#         prob_c.append(len(x1)/(len(x1)+len(x2)))
#         prob_c.append(len(x2)/(len(x1)+len(x2)))
        
#         output_list, z_sample_list, z_mean_list, z_logvar_list = self.forward(x1, x2)
        
#         Dist = 0
#         for j in range(self.c_Num):
#             mu_j = z_mean_list[j]
#             logvar = z_logvar_list[j]
#             z_sample =  z_sample_list[j]
#             dist_j = 0
#             diff = mu_j - z_sample
#             length = len(mu_j)
#             for i in range(self.z_dim):
#                 E = mu_j[:,i]
#                 logsig_i_minus_i = torch.cat((logvar[:,i,:i].T, logvar[:,i,i+1:].T)).T
#                 logsig_minus_minus = torch.cat((logvar[:,:i,:i].reshape(length,i*i), 
#                                                 logvar[:,:i,i+1:].reshape(length,i*(self.z_dim-i-1)),
#                                                 logvar[:,i+1:,:i].reshape(length,i*(self.z_dim-i-1)),
#                                                 logvar[:,i+1:,i+1:].reshape(length,(self.z_dim-i-1)*(self.z_dim-i-1))), 
#                                                 axis = 1).reshape(-1,self.z_dim-1, self.z_dim-1)
                
#                 diff_minus = torch.cat((diff[:,:i].T, diff[:,i+1:].T)).T
#                 do_E = mu_j[:,i] + torch.sum(torch.matmul(logsig_i_minus_i.reshape(-1,1,self.z_dim - 1), 
#                                                           torch.linalg.inv(logsig_minus_minus)).squeeze()*diff_minus, axis = 1)
#                 # absolute value for distance measure
#                 dist_j += torch.abs(do_E - E)
                
#             Dist += torch.mean(dist_j)*prob_c[j]
        
#         ## MSE reconstruction loss
#         reconstruction_error = F.mse_loss(output_list[0], x1) + F.mse_loss(output_list[1], x2)
            
#         loss = reconstruction_error + Dist
            
#         return loss, reconstruction_error, Dist
    
    


In [72]:
class doVAE_v2(nn.Module):

    def __init__(self,z_dim, c_dim = 2, in_channels=3):
        super(doVAE_v2, self).__init__()
        
        self.z_dim = z_dim
        self.c_dim = c_dim
        # encoder output dim: c_Num*z_dim for mu_z, c_Num*z_dim for sigma_z, c_Num for mu_pi, c_Num for sigma_mu
        self.model_encoder= nn.Sequential(
            # B,  32, 32, 32
            nn.Conv2d(in_channels, out_channels=32, kernel_size=4, stride=2, padding=1),          
            nn.BatchNorm2d(32),
            nn.ReLU(),
            # B,  32, 16, 16
            nn.Conv2d(32, out_channels=32, kernel_size=4, stride=2, padding=1),  
            nn.BatchNorm2d(32),
            nn.ReLU(),
            # B,  64,  8,  8
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1),  
            nn.BatchNorm2d(64),
            nn.ReLU(),
            # B,  64,  4,  4
            nn.Conv2d(64, 64, kernel_size=4, stride=2, padding=1),          
            nn.BatchNorm2d(64),
            nn.ReLU(),
            # B, 256,  1,  1
            nn.Conv2d(64, 256, kernel_size=4, stride=1),           
            nn.BatchNorm2d(256),
            nn.ReLU(),
            View((-1, 256*1*1)),                 # B, 256
            
            # for mu and logvar
            nn.Linear(256,  c_dim*(2*z_dim+2)),      
        )
        
            
        self.model_decoder = nn.Sequential(
            nn.Linear(z_dim, 256),               # B, 256
            View((-1, 256, 1, 1)),               # B, 256,  1,  1
            nn.ReLU(),
            nn.ConvTranspose2d(256, 64, 4),      # B,  64,  4,  4
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 64, 4, 2, 1), # B,  64,  8,  8
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 4, 2, 1), # B,  32, 16, 16
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.ConvTranspose2d(32, 32, 4, 2, 1), # B,  32, 32, 32
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.ConvTranspose2d(32, in_channels, 4, 2, 1),  # B, 3, 64, 64
        )
        
        hidden_dim = [32, 32]
        # parametrize P(z|c) ~ N(mu_c_prior, I)
        self.prior_layer= nn.Sequential(
            # B,  32, 32, 32
            nn.Linear(c_dim, hidden_dim[0]),          
            nn.Tanh(),
            # B,  32, 16, 16
            nn.Linear(hidden_dim[0], hidden_dim[1]),  
            nn.Tanh(),                 # B, 256
            # for mu and logvar
            nn.Linear(hidden_dim[1],  z_dim),      
        )

        self.weight_init()
        
    def weight_init(self):
        for block in self._modules:
            for m in self._modules[block]:
                kaiming_init(m)
        
                
    def encode(self, X):
        
        result = self.model_encoder(X)
        X_len = X.shape[0]
        mu = torch.zeros(X_len,self.c_dim * self.z_dim)
        logvar = torch.zeros(X_len,self.c_dim * self.z_dim)
        mu_pi = torch.zeros(X_len,self.c_dim)
        var_pi = torch.zeros(X_len,self.c_dim)
        for c in range(self.c_dim):
            mu[:,c*self.z_dim:(c+1)*self.z_dim] = result[:,c*self.z_dim : (c+1)*self.z_dim]
            logvar[:,c*self.z_dim:(c+1)*self.z_dim] = result[:,(self.c_dim+c)*self.z_dim : (self.c_dim+c+1)*self.z_dim]
            mu_pi[:,c] = result[:,2*self.c_dim*self.z_dim + c]
            var_pi[:,c] = result[:,2*self.c_dim*self.z_dim + self.c_dim + c]
                                
        
        return mu, logvar, mu_pi, var_pi
                                 
    
    def decode(self, z):
        
        return self.model_decoder(z)
    
    
    def generate(self, num_samples, current_device):                         
        z = torch.randn(num_samples, self.z_dim)
        z = z.to(current_device)
        samples = self.decode(z)
        return samples
    

    def reparameterize(self, mu, logvar,mu_pi,var_pi,label):
        X_len = mu.shape[0]
        z_all = torch.zeros(X_len, self.z_dim)
        mu_c = torch.zeros(X_len, self.z_dim)
        log_var_c = torch.zeros(X_len, self.z_dim)
        Pi = torch.zeros(X_len, self.c_dim)
        for c in range(self.c_dim):
            std = torch.exp(0.5 * logvar[:,c*self.z_dim:(c+1)*self.z_dim])
            eps = torch.randn_like(std)
            curr_z = eps * std + mu[:,c*self.z_dim:(c+1)*self.z_dim]
            
            std_pi = torch.exp(0.5 * var_pi[:,c])
            eps_pi = torch.randn_like(std_pi)
            curr_pi = eps_pi * std_pi + mu_pi[:,c]
            
            z_all += curr_z * curr_pi[:,None]
            Pi[:,c] = curr_pi
            # hard clustering for KL divergence
            mu_c +=  mu[:,c*self.z_dim:(c+1)*self.z_dim] * label[:,c][:,None]
            log_var_c += logvar[:,c*self.z_dim:(c+1)*self.z_dim]*label[:,c][:,None]
        
        return z_all, mu_c, log_var_c, Pi
    
    

    def forward(self, X,label):
        mu, logvar, mu_pi, var_pi = self.encode(X)
        z, mu_c, log_var_c, Pi = self.reparameterize(mu, logvar, mu_pi, var_pi,label)
        mu_c_prior = self.prior_layer(label.float())
        reconstructed_z = self.decode(z)
        return reconstructed_z, mu_c, log_var_c, mu_c_prior, Pi
    
    def calculate_loss(self, X, reconstructed_z, mu_c, log_var_c, mu_c_prior, Pi, label, pi_weight = 1.0, kl_weight = 0.5):
        """
        Loss term currently contains: reconstruction loss, classification loss for p(c|x)
        """
        # classification loss:
        criterion = nn.CrossEntropyLoss()
        pi_loss = criterion(Pi, label.float())
        
        # reconstruction loss:
        rec_loss = F.mse_loss(reconstructed_z, X)
        
        kld_loss = torch.mean(-0.5 * torch.sum(1 + log_var_c -(mu_c - mu_c_prior)**2 - log_var_c.exp(), dim = 1), dim = 0)
        
        loss = rec_loss + pi_weight * pi_loss + kl_weight * kld_loss
        
        return loss, rec_loss, pi_loss, kld_loss

In [73]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [88]:
model_type = 'v1'
if model_type == 'v1':
    CD_model = doVAE(z_dim=4)
elif model_type == 'v2':
    CD_model = doVAE_v2(z_dim=4)

In [95]:
train_samples = len(train_loader) * 128
learning_rate = 0.01*float(128)/256.
optimizer = torch.optim.SGD(CD_model.parameters(), lr=1e-3, 
                            momentum=0.9, weight_decay=5e-4)

In [ ]:
for epoch in tqdm(range(15)):
    CD_model.train()
    i = 0
    for x, target in train_loader:
        ### labels
        enc = OneHotEncoder(handle_unknown='ignore')
        enc.fit(target)
        target = torch.from_numpy(enc.transform(target).toarray())
        #####
        
        i += 1
        batch_time = AverageMeter()
        losses = AverageMeter()
        end = time.time()
        total_start = time.time()
        
        if model_type == 'v1':
            reconstructed_z, mu, logvar, Pi = CD_model(x)
            loss, rec_loss, pi_loss, kld_loss = CD_model.calculate_loss(x, reconstructed_z, mu, logvar, Pi, target)
        if model_type == 'v2':
            reconstructed_z, mu_c, log_var_c, mu_c_prior, Pi = CD_model(x,target)
            loss, rec_loss, pi_loss, kld_loss = CD_model.calculate_loss(x, reconstructed_z, mu_c, log_var_c, mu_c_prior, Pi, target)

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if i%15 == 0:
        
            reduced_loss = loss.data
            losses.update(float(reduced_loss), x.size(0))
            torch.cuda.synchronize()
            batch_time.update((time.time() - end)/15)
            end = time.time()

            print('Epoch: [{0}][{1}/{2}]\t'
                          'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                          'Loss {loss.val:.5f} ({loss.avg:.4f})\t'.format(
                           epoch, i, len(train_loader),
                           batch_time=batch_time,
                           loss=losses))
        

  0%|                                                    | 0/15 [00:00<?, ?it/s]

Epoch: [0][15/198]	Time 0.195 (0.195)	Loss 0.26293 (0.2629)	
Epoch: [0][30/198]	Time 0.185 (0.185)	Loss 0.26364 (0.2636)	
Epoch: [0][45/198]	Time 0.196 (0.196)	Loss 0.27257 (0.2726)	
Epoch: [0][60/198]	Time 0.213 (0.213)	Loss 0.25392 (0.2539)	
Epoch: [0][75/198]	Time 0.200 (0.200)	Loss 0.24980 (0.2498)	
Epoch: [0][90/198]	Time 0.245 (0.245)	Loss 0.26786 (0.2679)	
Epoch: [0][105/198]	Time 0.189 (0.189)	Loss 0.25181 (0.2518)	
Epoch: [0][120/198]	Time 0.191 (0.191)	Loss 0.24954 (0.2495)	
Epoch: [0][135/198]	Time 0.203 (0.203)	Loss 0.23914 (0.2391)	
Epoch: [0][150/198]	Time 0.201 (0.201)	Loss 0.25507 (0.2551)	
Epoch: [0][165/198]	Time 0.199 (0.199)	Loss 0.24802 (0.2480)	
Epoch: [0][180/198]	Time 0.186 (0.186)	Loss 0.25821 (0.2582)	
Epoch: [0][195/198]	Time 0.188 (0.188)	Loss 0.28102 (0.2810)	


  7%|██▋                                     | 1/15 [18:09<4:14:08, 1089.19s/it]

Epoch: [1][15/198]	Time 0.216 (0.216)	Loss 0.26770 (0.2677)	
Epoch: [1][30/198]	Time 0.252 (0.252)	Loss 0.26538 (0.2654)	
Epoch: [1][45/198]	Time 0.207 (0.207)	Loss 0.25098 (0.2510)	
Epoch: [1][60/198]	Time 0.206 (0.206)	Loss 0.25533 (0.2553)	
Epoch: [1][75/198]	Time 0.189 (0.189)	Loss 0.24685 (0.2468)	
Epoch: [1][90/198]	Time 0.188 (0.188)	Loss 0.26011 (0.2601)	
Epoch: [1][105/198]	Time 0.189 (0.189)	Loss 0.25668 (0.2567)	
Epoch: [1][120/198]	Time 0.184 (0.184)	Loss 0.26756 (0.2676)	
